In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import pickle
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import dataloader as dl

In [ ]:
train = dl.MyDataset(root_dir = "./",transform=transforms.Compose([transforms.ToTensor()]))
trainloader = torch.utils.data.DataLoader(train, batch_size=110,
                                      shuffle=True, num_workers=2)

In [ ]:
class detector(nn.Module):
    def __init__(self):
        super(detector, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=1, padding=1),  
            nn.ReLU(True),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),  
            nn.ReLU(True)
        )
        self.pool1 = nn.MaxPool2d(2, stride=2)  
    
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, stride=1, padding=1),  
            nn.ReLU(True),
            nn.Conv2d(32, 32, 3, stride=1, padding=1),  
            nn.ReLU(True)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, stride=1, padding=1),  
            nn.ReLU(True),
            nn.Conv2d(64, 64, 3, stride=1, padding=1),  
            nn.ReLU(True)
        )
        self.pool2 = nn.MaxPool2d(4,stride=4)
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(64,64,(7, 9),stride=1),
            nn.ReLU(True),
            nn.Conv2d(64,6,1,stride=1),
            nn.ReLU(True)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool1(x)  
        x = self.conv3(x)
        x = self.pool2(x)
        x = self.conv4(x)
        return x
    
model = detector().cuda(1)
def compute_loss(output,label):
    label = label.float()
    l_coord = 5
    l_nobj = 0.5
    sw_obj = label[:,0:1,:,:]
    sw_nobj = (label[:,0:1,:,:]==0).float()
    
    pred_loss_obj = torch.sum(nn.MSELoss(reduce=False)(output[:,0:1,:,:],label[:,0:1,:,:])*sw_obj)
    pred_loss_nobj = l_nobj*torch.sum(nn.MSELoss(reduce=False)(output[:,0:1,:,:],label[:,0:1,:,:])*sw_nobj)
    box_xy_loss = l_coord*torch.sum(nn.MSELoss(reduce=False)(output[:,1:3,:,:],label[:,1:3,:,:])*sw_obj)
    box_hw_loss = l_coord*torch.sum(nn.MSELoss(reduce=False)(torch.sqrt(output[:,3:5,:,:]),torch.sqrt(label[:,3:5,:,:]))*sw_obj)
    class_loss = torch.sum(nn.MSELoss(reduce=False)(output[:,5:6,:,:],label[:,5:6,:,:])*sw_obj)
    loss = pred_loss_obj + pred_loss_nobj + box_xy_loss + box_hw_loss + class_loss
    return loss

optimizer = torch.optim.Adam(model.parameters(), lr=0.01,
                             weight_decay=1e-5)

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    for data in trainloader:
        img, label = data
        img = Variable(img).cuda(1)
        label = Variable(label).cuda(1)

        # ===================forward=====================
        output = model(img)
        loss = compute_loss(output, label)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
      .format(epoch+1, 20, loss.data[0]))

In [ ]:
img,label = train[0]
img = Variable(img.unsqueeze(0).cuda(1))
# print img.data.shape
# label = Variable(label)
output = model(img)